<a href="https://colab.research.google.com/github/Danielakato/codelAB_bi/blob/main/DANKAY_BASEDEDATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A continuación, te proporcionaré el script en Python dividido en chunks para que puedas entender cada parte por separado.
Además, te mostraré cómo exportar cada tabla generada a una base de datos SQLite3.

# 1. Configuración Inicial y Creación de la Base de Datos
Este primer chunk configura la base de datos SQLite3 y crea la conexión a la misma.

In [9]:
!pip install faker  # Install the faker module

In [10]:
import sqlite3
import pandas as pd
import requests
import random
import uuid
from faker import Faker
import numpy as np
import shutil
from google.colab import drive

In [11]:
# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Inicializar Faker
fake = Faker()

# Crear la conexión a la base de datos SQLite3
conn = sqlite3.connect('financial_data.db2')

# Función para ejecutar consultas SQL
def execute_query(query, conn):
    with conn:
        conn.execute(query)

# Crear las tablas en SQLite3
execute_query('''CREATE TABLE IF NOT EXISTS customers (
                 customer_id TEXT PRIMARY KEY,
                 name TEXT,
                 address TEXT,
                 phone_number TEXT,
                 email TEXT);''', conn)

execute_query('''CREATE TABLE IF NOT EXISTS branches (
                 branch_id TEXT PRIMARY KEY,
                 branch_location TEXT,
                 manager_name TEXT,
                 contact_number TEXT);''', conn)

execute_query('''CREATE TABLE IF NOT EXISTS transaction_types (
                 transaction_type TEXT PRIMARY KEY,
                 description TEXT);''', conn)

execute_query('''CREATE TABLE IF NOT EXISTS transactions (
                 transaction_id TEXT PRIMARY KEY,
                 customer_id TEXT,
                 transaction_date TEXT,
                 transaction_amount REAL,
                 transaction_location TEXT,
                 transaction_type TEXT,
                 fraudulent INTEGER,
                 branch_id TEXT,
                 FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
                 FOREIGN KEY(transaction_type) REFERENCES transaction_types(transaction_type),
                 FOREIGN KEY(branch_id) REFERENCES branches(branch_id));''', conn)


# **2. Generar y Guardar la Tabla de Clientes**
Este chunk se encarga de obtener datos de la API de randomuser.me y de guardar la tabla de clientes en SQLite3.

In [20]:
# Función para obtener datos de clientes desde randomuser API
def get_random_users(num_users=300):
    url = f"https://randomuser.me/api/?results={num_users}&nat=us"
    response = requests.get(url)
    if response.status_code == 200:
        users = response.json()['results']
        return users
    else:
        print("Error fetching data from randomuser.me")
        return []

# Crear la tabla de clientes y guardarla en SQLite3
def create_customers_table(num_customers=300):
    users = get_random_users(num_customers)
    customers_data = {
        "customer_id": [str(uuid.uuid4()) for _ in range(num_customers)],
        "name": [f"{user['name']['first']} {user['name']['last']}" for user in users],
        "address": [f"{user['location']['street']['number']} {user['location']['street']['name']}, {user['location']['city']}, {user['location']['state']}, {user['location']['postcode']}" for user in users],
        "phone_number": [user['phone'] for user in users],
        "email": [user['email'] for user in users]
    }
    customers_df = pd.DataFrame(customers_data)
    customers_df.to_sql('customers', conn, if_exists='replace', index=False)
    return customers_df

# Crear la tabla de clientes
customers_df = create_customers_table(num_customers=100)

# **3. Generar y Guardar la Tabla de Sucursales**
Este chunk utiliza Faker para generar la tabla de sucursales y la guarda en SQLite3.

In [21]:
# Crear la tabla de sucursales y guardarla en SQLite3
def create_branches_table(num_branches=10):
    branch_data = {
        "branch_id": [str(uuid.uuid4()) for _ in range(num_branches)],
        "branch_location": [fake.city() for _ in range(num_branches)],
        "manager_name": [fake.name() for _ in range(num_branches)],
        "contact_number": [fake.phone_number() for _ in range(num_branches)]
    }
    branches_df = pd.DataFrame(branch_data)
    branches_df.to_sql('branches', conn, if_exists='replace', index=False)
    return branches_df

# Crear la tabla de sucursales
branches_df = create_branches_table(num_branches=20)

# **4. Generar y Guardar la Tabla de Tipos de Transacciones**
Este chunk crea la tabla de tipos de transacciones y la guarda en SQLite3.

In [22]:
# Crear la tabla de tipos de transacciones y guardarla en SQLite3
def create_transaction_types_table():
    transaction_types_data = {
        "transaction_type": ["online", "in-store"],
        "description": ["Transaction made online via the internet",
                        "Transaction made at a physical store location"]
    }
    transaction_types_df = pd.DataFrame(transaction_types_data)
    transaction_types_df.to_sql('transaction_types', conn, if_exists='replace', index=False)
    return transaction_types_df

# Crear la tabla de tipos de transacciones
transaction_types_df = create_transaction_types_table()

# **5. Generar y Guardar la Tabla de Transacciones**
Este chunk genera la tabla de transacciones, incluyendo las transacciones fraudulentas, y la guarda en SQLite3.

In [23]:
# Crear la tabla de transacciones y guardarla en SQLite3
def create_transactions_table(customers_df, branches_df, num_transactions=100):
    transaction_data = {
        "transaction_id": [str(uuid.uuid4()) for _ in range(num_transactions)],
        "customer_id": [random.choice(customers_df['customer_id']) for _ in range(num_transactions)],
        "transaction_date": [fake.date_time_this_year().isoformat() for _ in range(num_transactions)],
        "transaction_amount": [round(random.uniform(10.0, 1000.0), 2) for _ in range(num_transactions)],
        "transaction_location": [fake.city() for _ in range(num_transactions)],
        "transaction_type": [random.choice(["online", "in-store"]) for _ in range(num_transactions)],
        "fraudulent": [0] * num_transactions
    }
    transactions_df = pd.DataFrame(transaction_data)

    # Introducir transacciones fraudulentas
    n_fraud = 10  # Número de transacciones fraudulentas
    fraud_indices = np.random.choice(transactions_df.index, n_fraud, replace=False)
    transactions_df.loc[fraud_indices, 'fraudulent'] = 1
    transactions_df.loc[fraud_indices, 'transaction_amount'] = [round(random.uniform(1000.0, 5000.0), 2) for _ in range(n_fraud)]
    transactions_df.loc[fraud_indices, 'transaction_type'] = "online"

    # Asignar branch_id solo para transacciones "in-store"
    in_store_indices = transactions_df[transactions_df['transaction_type'] == 'in-store'].index
    transactions_df.loc[in_store_indices, 'branch_id'] = np.random.choice(branches_df['branch_id'], size=len(in_store_indices))

    # Para transacciones online, branch_id se puede establecer en None
    transactions_df['branch_id'].fillna('None', inplace=True)

    transactions_df.to_sql('transactions', conn, if_exists='replace', index=False)
    return transactions_df

# Crear la tabla de transacciones
transactions_df = create_transactions_table(customers_df, branches_df, num_transactions=500)


# **6. Verificación y Cierre de la Conexión**
Este chunk verifica que las tablas se crearon correctamente y cierra la conexión con la base de datos.

In [24]:
# Verificar la creación de tablas en SQLite3
print("Tablas en la base de datos SQLite:")
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print(tables)

# Cerrar la conexión
conn.close()

Tablas en la base de datos SQLite:
                name
0          customers
1           branches
2  transaction_types
3       transactions


In [27]:
# Ruta en Google Drive donde deseas guardar la base de datos
drive_db_path = '/content/drive/My Drive/financial_data.db2'

# Copiar el archivo de la base de datos a Google Drive
shutil.copy('financial_data.db2', drive_db_path)

print(f'Base de datos guardada en: {drive_db_path}')

Base de datos guardada en: /content/drive/My Drive/financial_data.db2
